In [1]:
import os



In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import load_model
from pathlib import Path
import numpy as np

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
#dataset pre-processing
BATCH_SIZE = 64
img_size = (300,300)
#training data generator
traingen = ImageDataGenerator(       rotation_range=90, 
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5, 
                                     height_shift_range=0.5,
                                     horizontal_flip=True, 
                                     vertical_flip=True,
                                     validation_split=0.15,
                                     preprocessing_function=preprocess_input)

#test dataset generator
testgen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [6]:
#dataset path for training and testing data
path_to_training_data = 'E:/ambience_mode/dataset/train'
path_to_testing_data = 'E:/ambience_mode/dataset/test'

#class labels
labels = os.listdir(path_to_training_data)
#training data
training_data = traingen.flow_from_directory(  path_to_training_data,
                                               target_size=img_size,
                                               class_mode='categorical',
                                               classes=labels,
                                               subset='training',
                                               batch_size=BATCH_SIZE, 
                                               shuffle=True,
                                               seed=42)
#validation data
validation_data = traingen.flow_from_directory(path_to_training_data,
                                               target_size=img_size,
                                               class_mode='categorical',
                                               classes=labels,
                                               subset='validation',
                                               batch_size=BATCH_SIZE, 
                                               shuffle=True,
                                               seed=42)

#testing data
testing_data = testgen.flow_from_directory(    path_to_testing_data,
                                               target_size=img_size,
                                               class_mode=None,
                                               classes=labels,
                                               batch_size=1, 
                                               shuffle=False,
                                               seed=42)

Found 1275 images belonging to 5 classes.
Found 223 images belonging to 5 classes.
Found 30 images belonging to 5 classes.


In [7]:
def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):
    """
    Compiles a model integrated with VGG16 pretrained layers
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """
    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = VGG16(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(16, activation='relu')(top_model)
    top_model = Dense(32, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [11]:
input_shape = (300, 300, 3)
optim_2 = Adam(learning_rate=0.0001)
n_classes=5

n_steps = training_data.samples // BATCH_SIZE
n_val_steps = validation_data.samples // BATCH_SIZE
n_epochs = 20

# First we'll train the model without Fine-tuning
vgg_model_ft = create_model(input_shape, n_classes, optim_2, fine_tune=0)


In [12]:
vgg_model_ft.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0   

^C
Note: you may need to restart the kernel to use updated packages.


In [14]:
from livelossplot import PlotLossesKeras
from livelossplot.keras import PlotLossesCallback

plot_loss_2 = PlotLossesCallback()

# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='E:/ambience_mode/new metrics/tl_model_v1.weights.best.hdf5',
                                  save_best_only=True,
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')

ModuleNotFoundError: No module named 'livelossplot'

In [ ]:
vgg_ft_history = vgg_model_ft.fit(training_data,
                                  batch_size=BATCH_SIZE,
                                  epochs=n_epochs,
                                  validation_data=validation_data,
                                  steps_per_epoch=n_steps, 
                                  validation_steps=n_val_steps,
                                  verbose=1)

Epoch 1/20
